# Data Processing

In [30]:
# !pip install keras_applications

In [31]:
import numpy as np
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import cv2
warnings.filterwarnings("ignore")
from numpy.random import permutation
np.random.seed(2016)
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPool2D # GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import utils
from keras.models import model_from_json
from sklearn.metrics import log_loss, confusion_matrix
from keras import regularizers
import h5py

from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import confusion_matrix

from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, \
    GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Conv2D
from keras.models import Model
# from keras.engine.topology import get_source_inputs
from keras.losses import CategoricalCrossentropy
from keras.utils import get_source_inputs, plot_model
# from depthwise_conv2d import DepthwiseConvolution2D

In [32]:
use_cache = 1
dataset_path = r'../Dataset/v2_cam1_cam2'

np.random.seed(1234)
warnings.filterwarnings("ignore")

In [33]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(src=img, dsize=(224, 224), interpolation=cv2.INTER_LINEAR) / 255.0
    return resized

In [34]:
def load_train(num_classes=10):
    train_dir_cam1 = dataset_path + '/Camera 1/train/'
    train_dir_cam2 = dataset_path + '/Camera 2/train/'

    X_train = []
    Y_train = []

    print('Read train images')

    for label in range(num_classes):
        class_label = 'c' + str(label)  # Example: c0
        class_dir_cam1 = train_dir_cam1 + class_label
        class_dir_cam2 = train_dir_cam2 + class_label

        x1 = os.listdir(class_dir_cam1)
        x2 = os.listdir(class_dir_cam2)

        for i in range (0,len(x1)):
            print(f"Reading image of camera 1 - class {class_label}: {i}/{len(x1)}")
            img = get_im_cv2(class_dir_cam1 + '/' + x1[i])
            X_train.append(img)
            Y_train.append(label)

        for i in range (0,len(x2)):
            print(f"Reading image of camera 2 - class {class_label}: {i}/{len(x2)}")
            img = get_im_cv2(class_dir_cam2 + '/' + x2[i])
            X_train.append(img)
            Y_train.append(label)


    return X_train, Y_train

In [35]:
def load_valid(num_classes=10):
    test_dir_cam1 = dataset_path + '/Camera 1/test/'
    test_dir_cam2 = dataset_path + '/Camera 2/test/'

    X_valid = []
    Y_valid = []

    print('Read test images')

    for label in range(num_classes):
        class_label = 'c' + str(label)  # Example: c0
        class_dir_cam1 = test_dir_cam1 + class_label
        class_dir_cam2 = test_dir_cam2 + class_label

        x1 = os.listdir(class_dir_cam1)
        x2 = os.listdir(class_dir_cam2)

        for i in range (0,len(x1)):
            print(f"Reading image of camera 1 - class {class_label}: {i}/{len(x1)}")
            img = get_im_cv2(class_dir_cam1 + '/' + x1[i])
            X_valid.append(img)
            Y_valid.append(label)

        for i in range (0,len(x2)):
            print(f"Reading image of camera 2 - class {class_label}: {i}/{len(x2)}")
            img = get_im_cv2(class_dir_cam2 + '/' + x2[i])
            X_valid.append(img)
            Y_valid.append(label)

    return X_valid, Y_valid

In [36]:
def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')

In [37]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

In [38]:
def read_and_normalize_train_data():
    cache_path = dataset_path + '/cache/train_v1.dat'
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target= load_train()
        cache_data((train_data, train_target), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target) = restore_data(cache_path)
    
    print('Convert to numpy...')
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    
    print('Reshape...')
    train_data = train_data.transpose((0, 1, 2, 3))

    # Normalise the train data
    print('Convert to float...')
    train_data = train_data.astype('float16')
    mean_pixel = [80.857, 81.106, 82.928]
    
    print('Substract 0...')
    train_data[:, :, :, 0] -= mean_pixel[0]
    
    print('Substract 1...')
    train_data[:, :, :, 1] -= mean_pixel[1]

    print('Substract 2...')
    train_data[:, :, :, 2] -= mean_pixel[2]

    train_target = utils.to_categorical(train_target, 10)
    
    # Shuffle experiment START !!
    perm = permutation(len(train_target))
    train_data = train_data[perm]
    train_target = train_target[perm]
    # Shuffle experiment END !!
    
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    print('Target shape:', train_target.shape)
    print(train_target.shape[0], 'target samples')
    return train_data, train_target

In [39]:
def read_and_normalize_test_data():
    start_time = time.time()
    cache_path = dataset_path + '/cache/test_v1.dat'

    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_target = load_valid()
        cache_data((test_data, test_target ), cache_path)
    else:
        print('Restore test from cache [{}]!')
        (test_data, test_target) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.transpose((0, 1, 2, 3))

    # Normalise the test data data

    test_data = test_data.astype('float16')
    mean_pixel = [80.857, 81.106, 82.928]

    test_data[:, :, :, 0] -= mean_pixel[0]

    test_data[:, :, :, 1] -= mean_pixel[1]

    test_data[:, :, :, 2] -= mean_pixel[2]

    test_target = utils.to_categorical(test_target, 10)
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    print('Target shape:', test_target.shape)
    print(test_target.shape[0], 'target samples')
    print('Read and process test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return test_data, test_target

In [40]:
X_train, Y_train = read_and_normalize_train_data()
X_valid, Y_valid = read_and_normalize_test_data()

Read train images
Reading image of camera 1 - class c0: 0/2440
Reading image of camera 1 - class c0: 1/2440
Reading image of camera 1 - class c0: 2/2440
Reading image of camera 1 - class c0: 3/2440
Reading image of camera 1 - class c0: 4/2440
Reading image of camera 1 - class c0: 5/2440
Reading image of camera 1 - class c0: 6/2440
Reading image of camera 1 - class c0: 7/2440
Reading image of camera 1 - class c0: 8/2440
Reading image of camera 1 - class c0: 9/2440
Reading image of camera 1 - class c0: 10/2440
Reading image of camera 1 - class c0: 11/2440
Reading image of camera 1 - class c0: 12/2440
Reading image of camera 1 - class c0: 13/2440
Reading image of camera 1 - class c0: 14/2440
Reading image of camera 1 - class c0: 15/2440
Reading image of camera 1 - class c0: 16/2440
Reading image of camera 1 - class c0: 17/2440
Reading image of camera 1 - class c0: 18/2440
Reading image of camera 1 - class c0: 19/2440
Reading image of camera 1 - class c0: 20/2440
Reading image of camera 1 

# Run Model